In [171]:
from pandasql import sqldf
import pandas as pd
import WazeRouteCalculator as wrc
import numpy as np
from geopy.distance import geodesic
import openpyxl

In [5]:
df_barrios = pd.read_csv('barrios.csv')
df_dispositivos = pd.read_csv('dispositivos.csv')
df_clientes = pd.read_csv('clientes.csv')

In [6]:
df_barrios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   codigo  337 non-null    int64 
 1   nombre  337 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [7]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93446 entries, 0 to 93445
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   num_doc          93446 non-null  int64  
 1   tipo_doc         93446 non-null  int64  
 2   canal            93446 non-null  object 
 3   cod_dispositivo  93446 non-null  int64  
 4   num_trx          93446 non-null  int64  
 5   mnt_total_trx    93446 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 4.3+ MB


In [8]:
df_dispositivos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37284 entries, 0 to 37283
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tipo       37284 non-null  object 
 1   codigo     37284 non-null  int64  
 2   latitud    37284 non-null  float64
 3   longitud   37284 non-null  float64
 4   id_barrio  37284 non-null  int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 1.4+ MB


In [11]:
df_barrios

,codigo,nombre
0,737998257,La Campina
1,737999435,Guayaquil
2,737998219,Vipasa
3,737996837,Lourdes
4,737997031,Villanueva
...,...,...
332,737998799,Ignacio Rengifo
333,738000408,Prados del Limonar
334,737998040,Pizamos III-Las Dalias
335,737998078,Brisas de Mayo


In [24]:
df_dispositivos['dup_validate'] = df_dispositivos.duplicated(subset='codigo', keep='first')
#df_dispositivos[df_dispositivos['dup_validate'] == True]
df_dispositivos[df_dispositivos['codigo'].isin([745,747])]

#Hay que cruzar incluyendo el id_barrio también en la llave, dado que no son únicos los codigos de los dispositivos
#Se debe corregir el diagrama dado que no sería una relación de uno a varios sino de varios a varios

,tipo,codigo,latitud,longitud,id_barrio,dup_validate
6745,SAI,745,3.414325,-76.547986,737998095,False
6746,SAI,747,3.485807,-76.516292,737998174,False
6776,DISPENSADOR,745,3.451204,-76.532942,737998832,True
6778,DISPENSADOR,747,3.451204,-76.532942,737998832,True


In [17]:
df_clientes

,num_doc,tipo_doc,canal,cod_dispositivo,num_trx,mnt_total_trx
0,6861792605025121524,1,POS,14812028,3,22218042.0
1,1552970090928921643,1,POS,14388938,3,21583407.0
2,-4075904842556875666,1,POS,12370698,1,6185349.0
3,3539339627033221084,1,POS,10313583,2,10313583.0
4,5595554501572542610,1,POS,14358071,4,28716142.0
...,...,...,...,...,...,...
93441,-5638568367379363305,2,POS,13809496,1,6904748.0
93442,-6859292209449934659,2,DISPENSADOR,1116,1,558.0
93443,-1762474661445594290,2,SAI,745,1,372.5
93444,-2708241712915270749,2,DISPENSADOR,4090,1,2045.0


In [27]:
df_dispositivos_clean = (
    df_dispositivos.copy()
    .query('dup_validate != True')
    .drop(['dup_validate'], axis=1)
)

In [35]:
df_tmp_merge = df_dispositivos_clean.merge(df_barrios, how='left', left_on=['id_barrio'], right_on=['codigo'], indicator=True)
df_tmp_merge = df_tmp_merge.drop(['codigo_y', '_merge'], axis=1)
df_tmp_merge = df_tmp_merge.rename(columns={'codigo_x':'codigo'})

In [38]:
df_sabana_datos = df_clientes.merge(df_tmp_merge, how='left', left_on=['cod_dispositivo'], right_on=['codigo'], indicator=True)

In [65]:
query_q1 = """
SELECT
    *
FROM(
    SELECT
        num_doc,
        COUNT(DISTINCT id_barrio) as barrios_con_trx
    FROM 
        df_sabana_datos
    WHERE 
        num_trx > 0
    GROUP BY 
        num_doc
)
WHERE 
    barrios_con_trx >= 5
ORDER BY 
    barrios_con_trx desc
"""

In [66]:
sqldf(query_q1, env=None)

,num_doc,barrios_con_trx
0,-9218698523982046880,8
1,-9210147727599394517,7
2,-9202442483227019464,6
3,-9202223027335988936,7
4,-9201713395891677864,5
...,...,...
3435,9201395475332460529,13
3436,9206935107044370594,28
3437,9207292192995699304,9
3438,9219111613418646473,15


In [92]:
query_q2 = """
SELECT
    nombre as nombre_barrio,
    COUNT(DISTINCT marcacion_pac) q_clientes_unicos_pac,
    COUNT(DISTINCT num_doc) q_clientes_unicos_barrio
FROM (   
    SELECT
        *,
        CASE
          WHEN canal = 'PAC'
          THEN num_doc
          ELSE null END as marcacion_pac
    FROM
        df_sabana_datos)
GROUP BY 
    nombre
ORDER BY 
    q_clientes_unicos_pac desc
LIMIT 6
"""

In [93]:
#En esta pregunta debemos considerar dos escenarios, uno que es el más acercado a la realidad donde se debería concatenar el tipo de documento
#con el numero de documento y ahí si con esa llave unica por cliente hacer el COUNT DISTINCT, el otro escenario más simplista considera nada más el numero de documento

sqldf(query_q2, env=None)

,nombre_barrio,q_clientes_unicos_pac,q_clientes_unicos_barrio
0,San Nicolás,5,820
1,San Pedro,4,2009
2,Colseguros Andes,4,345
3,Prados del Norte,1,786
4,Las Delicias,1,363
5,El Troncal,1,457


In [95]:
#Para esto debemos construir una tabla donde pivoteemos el numero de transacciones por cada canal (columnas) y construir la columna del num total de transacciones por cliente
#Luego calculamos el % de cada canal sobre el total de transacciones y ahí si filtramos el canal que más usa (+51% de uso) (Hacerlo para el numero de trx y para el monto para ver la diferencia)

query_q3_1 = """
SELECT
    num_doc,
    SUM(num_trx) total_trx
FROM
    df_sabana_datos
GROUP BY
    num_doc
"""
df_q3_1 = sqldf(query_q3_1, env=None)

In [100]:
query_q3_2 = """
SELECT
    num_doc,
    canal,
    SUM(num_trx) trx_canal
FROM
    df_sabana_datos
GROUP BY
    num_doc,
    canal
"""
df_q3_2 = sqldf(query_q3_2, env=None).merge(df_q3_1, how='left', on=['num_doc'])

In [104]:
df_q3_2['proporcion_uso_canal'] = df_q3_2['trx_canal'] / df_q3_2['total_trx']

,num_doc,canal,trx_canal,total_trx,proporcion_uso_canal
1,-9218698523982046880,MF,6,14,0.428571
6,-9210147727599394517,SAI,2,35,0.057143
8,-9202442483227019464,DISPENSADOR,2,8,0.250000
10,-9202223027335988936,CB,5,12,0.416667
11,-9202223027335988936,DISPENSADOR,3,12,0.250000
...,...,...,...,...,...
20968,9219111613418646473,CB,4,37,0.108108
20970,9219111613418646473,POS,11,37,0.297297
20972,9220077217609243452,CB,7,111,0.063063
20974,9220077217609243452,POS,14,111,0.126126


In [133]:
#q3 final
dicc_clientes_canales = {}
for i in df_q3_2['num_doc'].unique():
    df_tmp = df_q3_2[df_q3_2['num_doc'] == i].sort_values(['proporcion_uso_canal'], ascending =[False])
    tmp_list = []
    tmp_sum = 0
    for row in range(0,len(df_tmp)):
        tmp_list.append(df_tmp.iloc[row]['canal'])
        tmp_sum += df_tmp.iloc[row]['proporcion_uso_canal']
        if tmp_sum >= 0.51:
            break
    dicc_clientes_canales[i] = tmp_list

In [134]:
dicc_clientes_canales

{-9218698523982046880: ['CB'],
 -9214605575224426554: ['SAI'],
 -9211522780831135515: ['SAI'],
 -9210956249921547631: ['POS'],
 -9210147727599394517: ['DISPENSADOR'],
 -9204167197570414556: ['CB'],
 -9202442483227019464: ['POS'],
 -9202223027335988936: ['CB', 'MF'],
 -9201713395891677864: ['DISPENSADOR'],
 -9201641477436017839: ['SAI'],
 -9191710335355420570: ['POS'],
 -9186693395875698210: ['POS', 'MF'],
 -9185850207136741917: ['POS'],
 -9183474780083267196: ['SAI'],
 -9181925865194571665: ['DISPENSADOR'],
 -9180307750932599349: ['DISPENSADOR'],
 -9177183552582173731: ['DISPENSADOR'],
 -9176036693048770286: ['DISPENSADOR'],
 -9175745549450915199: ['DISPENSADOR'],
 -9171373680224819785: ['CB'],
 -9171065409699835768: ['SAI'],
 -9167539163478279244: ['CB'],
 -9166547876666667911: ['SAI'],
 -9162351395024733017: ['SAI'],
 -9161274492736720159: ['CB'],
 -9160713943500174365: ['CB', 'DISPENSADOR'],
 -9159441166222233466: ['POS'],
 -9157925088953969578: ['DISPENSADOR'],
 -915541073214171130

In [136]:
#q4
#Para esto primero aislamos el barrio panamericano e identificamos los dispositivos unicos junto con sus ubicaciones
query_q4_1 = """
SELECT DISTINCT
    canal,
    cod_dispositivo,
    latitud,
    longitud
FROM
    df_sabana_datos
WHERE
    nombre LIKE '%Panamericano%'
"""
df_q4_1 = sqldf(query_q4_1, env=None)

In [143]:
df_q4_1['loc'] = df_q4_1['latitud'].astype(str) + ", " + df_q4_1['longitud'].astype(str)

In [154]:
def dist_dict(df):
  dist_dict = {}
  for i in range(0,len(df)):

    dist_dict[df['cod_dispositivo'].iloc[i]] = df['loc'].iloc[i]
  
  return dist_dict

In [180]:
def dist_matrix(dicc):
    keys = list(dicc.keys())
    lista1 = keys.copy()
    lista2 = keys.copy()
    values = np.zeros((len(lista1),len(lista2)))
    values1 = np.zeros((len(lista1),len(lista2)))

    for loc0, value0 in enumerate(dicc.values()):
        for loc1, value1 in enumerate(dicc.values()):
        
            if loc0 == loc1:
                route_dist = 0
                linear_dist = 0        
            else:
                from_address = value0
                to_address = value1

                route = wrc.WazeRouteCalculator(from_address, to_address)
                linear_dist = geodesic(from_address, to_address).kilometers
                try:
                    route_time, route_dist = route.calc_route_info()
                except wrc.WRCError as err:
                    print(err)

        
            values[(loc0),(loc1)] = route_dist
            values1[(loc0),(loc1)] = linear_dist

    df = pd.DataFrame(data=values, index=lista1, columns=lista2)
    df1 = pd.DataFrame(data=values1, index=lista1, columns=lista2)
    return values, df, values1, df1

In [181]:
loc_dict = dist_dict(df_q4_1)
dist_matrix_waze, dist_df_waze, dist_matriz_linear, dist_df_linear = dist_matrix(loc_dict)

empty response
empty response
empty response
empty response


In [162]:
dist_df_waze

,14994693,15598709,13977004,11075827,14907158,11646205,14805709,12200812,12690186,15186059,14614143,1979425573,15616816,13158795,13027248
14994693,0.000,0.829,0.768,0.014,0.688,0.722,1.220,1.380,0.460,0.743,0.718,1.358,0.098,1.162,0.718
15598709,0.112,0.000,0.044,0.126,1.956,1.282,1.066,1.226,1.277,0.049,1.278,1.204,0.211,1.008,1.278
13977004,0.207,0.005,0.000,0.221,0.008,0.017,1.013,1.173,0.264,0.000,0.013,1.151,0.306,0.955,0.013
11075827,0.014,0.828,0.767,0.000,0.687,0.721,1.220,1.380,0.459,0.742,0.717,1.358,0.008,1.162,0.717
14907158,0.251,0.107,0.038,0.265,0.000,0.009,0.961,0.013,0.255,0.021,0.009,0.046,0.350,0.903,0.009
11646205,0.250,0.106,0.037,0.264,0.001,0.000,0.961,0.013,1.415,0.020,0.004,1.099,0.349,0.903,0.004
14805709,0.466,0.322,0.261,0.480,2.310,0.215,0.000,0.159,1.631,0.236,0.211,0.137,0.565,0.040,0.211
12200812,0.282,0.138,0.077,0.296,0.024,0.000,0.923,0.000,1.447,0.052,0.024,0.022,0.381,0.865,0.024
12690186,0.486,0.342,0.281,0.503,0.201,0.235,0.442,0.469,0.000,0.256,0.231,0.447,0.588,0.482,0.231
15186059,0.213,0.000,0.038,0.227,0.033,0.034,0.998,1.158,0.264,0.000,0.030,1.136,0.312,0.940,0.030


In [182]:
dist_df_linear

,14994693,15598709,13977004,11075827,14907158,11646205,14805709,12200812,12690186,15186059,14614143,1979425573,15616816,13158795,13027248
14994693,0.000000,0.143210,0.207073,0.014283,0.251381,0.249494,0.467195,0.306100,0.331175,0.229454,0.253789,0.328273,0.080188,0.524890,0.253789
15598709,0.143210,0.000000,0.063873,0.157483,0.108299,0.106301,0.323991,0.162908,0.238986,0.086258,0.110596,0.185081,0.223375,0.381685,0.110596
13977004,0.207073,0.063873,0.000000,0.221349,0.044984,0.042429,0.260122,0.099036,0.217850,0.022385,0.046724,0.121208,0.287245,0.317816,0.046724
11075827,0.014283,0.157483,0.221349,0.000000,0.265635,0.263772,0.481471,0.320377,0.341952,0.243731,0.268066,0.342550,0.065906,0.539165,0.268066
14907158,0.251381,0.108299,0.044984,0.265635,0.000000,0.009597,0.216154,0.055801,0.204247,0.023577,0.009862,0.077729,0.331468,0.273799,0.009862
11646205,0.249494,0.106301,0.042429,0.263772,0.009597,0.000000,0.217701,0.056608,0.213711,0.020043,0.004295,0.078780,0.329670,0.275397,0.004295
14805709,0.467195,0.323991,0.260122,0.481471,0.216154,0.217701,0.000000,0.161102,0.301833,0.237740,0.213407,0.138932,0.547366,0.057696,0.213407
12200812,0.306100,0.162908,0.099036,0.320377,0.055801,0.056608,0.161102,0.000000,0.220729,0.076651,0.052313,0.022173,0.386277,0.218798,0.052313
12690186,0.331175,0.238986,0.217850,0.341952,0.204247,0.213711,0.301833,0.220729,0.000000,0.214635,0.213757,0.227180,0.394952,0.344870,0.213757
15186059,0.229454,0.086258,0.022385,0.243731,0.023577,0.020043,0.237740,0.076651,0.214635,0.000000,0.024338,0.098823,0.309628,0.295436,0.024338


In [191]:
df_result = pd.DataFrame([])
for codigo in df_q4_1['cod_dispositivo']:
    #print(codigo)
    #Tomamos todos los valores menos el codigo a buscar
    aux = df_q4_1[df_q4_1['cod_dispositivo']!=codigo]
    #Tomamos el código a buscar
    row = df_q4_1[df_q4_1['cod_dispositivo']==codigo]
    #Asignamos una columna con el valor del código para tener en la misma fila el código consultado.
    aux['codigo_consultado'] = codigo
    aux['loc_consultado'] = row.iloc[0]['loc']
    
    #definimos la columna distancia en la que vamos a asignar el resultado de haber aplicado la función al df.
    aux['distancia'] = aux.apply(lambda fila: geodesic(row.iloc[0]['loc'], fila['loc']).kilometers,
                                axis=1)
    #Agregamos el dataframe reducido a nuestro dataframe resultante.
    df_result = pd.concat([df_result, aux])

/var/folders/yt/mbjmcc4551d674wt7vkrk3740000gn/T/ipykernel_8198/2931325889.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux['codigo_consultado'] = codigo
/var/folders/yt/mbjmcc4551d674wt7vkrk3740000gn/T/ipykernel_8198/2931325889.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux['loc_consultado'] = row.iloc[0]['loc']
/var/folders/yt/mbjmcc4551d674wt7vkrk3740000gn/T/ipykernel_8198/2931325889.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [207]:
df_q4_2 = df_result.sort_values(['distancia'], ascending =[False]).head(20)

In [208]:
df_q4_2['dup_validation'] = df_q4_2.duplicated(subset='distancia', keep='first')

In [209]:
df_q4_2 = df_q4_2[df_q4_2['dup_validation'] == False].reset_index(drop=True)

In [211]:
df_q4_2

,canal,cod_dispositivo,latitud,longitud,loc,codigo_consultado,loc_consultado,distancia,dup_validation
0,POS,13158795,3.416580,-76.532074,"3.416580484, -76.53207354",15616816,"3.41138520464, -76.5337825036",0.605059,False
1,POS,14805709,3.416084,-76.532232,"3.41608366616, -76.5322321723",15616816,"3.41138520464, -76.5337825036",0.547366,False
2,POS,13158795,3.416580,-76.532074,"3.416580484, -76.53207354",11075827,"3.41194761862, -76.5335861823",0.539165,False
3,POS,13158795,3.416580,-76.532074,"3.416580484, -76.53207354",14994693,"3.41206905851, -76.5335424073",0.524890,False
4,POS,14805709,3.416084,-76.532232,"3.41608366616, -76.5322321723",11075827,"3.41194761862, -76.5335861823",0.481471,False
5,POS,14994693,3.412069,-76.533542,"3.41206905851, -76.5335424073",14805709,"3.41608366616, -76.5322321723",0.467195,False
6,POS,15616816,3.411385,-76.533783,"3.41138520464, -76.5337825036",1979425573,"3.414886, -76.53261",0.408450,False
7,POS,15616816,3.411385,-76.533783,"3.41138520464, -76.5337825036",12690186,"3.41484784983, -76.5346540488",0.394952,False
8,POS,12200812,3.414696,-76.532673,"3.414695895, -76.53267347",15616816,"3.41138520464, -76.5337825036",0.386277,False
9,POS,15598709,3.413303,-76.533150,"3.41330275619, -76.5331502918",13158795,"3.416580484, -76.53207354",0.381685,False
